# XGBoost (Regression)

---

**Источники:**

[XGBoost](https://neerc.ifmo.ru/wiki/index.php?title=XGBoost)

[Виды ансамблей](https://neerc.ifmo.ru/wiki/index.php?title=%D0%92%D0%B8%D0%B4%D1%8B_%D0%B0%D0%BD%D1%81%D0%B0%D0%BC%D0%B1%D0%BB%D0%B5%D0%B9)

[XGBoost Parameters](https://xgboost.readthedocs.io/en/latest/parameter.html)

[Эвристический алгоритм](https://ru.wikipedia.org/wiki/%D0%AD%D0%B2%D1%80%D0%B8%D1%81%D1%82%D0%B8%D1%87%D0%B5%D1%81%D0%BA%D0%B8%D0%B9_%D0%B0%D0%BB%D0%B3%D0%BE%D1%80%D0%B8%D1%82%D0%BC#:~:text=%D0%AD%D0%B2%D1%80%D0%B8%D1%81%D1%82%D0%B8%D1%87%D0%B5%D1%81%D0%BA%D0%B8%D0%B9%20%D0%B0%D0%BB%D0%B3%D0%BE%D1%80%D0%B8%D1%82%D0%BC%20(%D1%8D%D0%B2%D1%80%D0%B8%D1%81%D1%82%D0%B8%D0%BA%D0%B0)%20%E2%80%94%20%D0%B0%D0%BB%D0%B3%D0%BE%D1%80%D0%B8%D1%82%D0%BC,%D1%80%D0%B5%D1%88%D0%B5%D0%BD%D0%B8%D0%B5%20%D0%BD%D0%B5%20%D0%BC%D0%BE%D0%B6%D0%B5%D1%82%20%D0%B1%D1%8B%D1%82%D1%8C%20%D0%BD%D0%B0%D0%B9%D0%B4%D0%B5%D0%BD%D0%BE.)

---

**Одна из самых популярных и эффективных реализаций алгоритма градиентного бустинга на деревьях на 2019-й год.**


В основе XGBoost лежит алгоритм градиентного бустинга деревьев решений. 

**Градиентный бустинг** — это техника машинного обучения для задач классификации и регрессии, которая строит модель предсказания в форме ансамбля слабых предсказывающих моделей, обычно деревьев решений. Обучение ансамбля проводится последовательно в отличие, например от бэггинга. 

**Ансамбль алгоритмов (методов)** — метод, который использует несколько обучающих алгоритмов с целью получения лучшей эффективности прогнозирования, чем можно было бы получить от каждого обучающего алгоритма по отдельности.

На каждой итерации вычисляются отклонения предсказаний уже обученного ансамбля на обучающей выборке. Следующая модель, которая будет добавлена в ансамбль будет предсказывать эти отклонения. Таким образом, добавив предсказания нового дерева к предсказаниям обученного ансамбля мы можем уменьшить среднее отклонение модели, котрое является таргетом оптимизационной задачи. Новые деревья добавляются в ансамбль до тех пор, пока ошибка уменьшается, либо пока не выполняется одно из правил "ранней остановки".


XGBoost поддерживает все возможности таких библиотек как scikit-learn с возможностью добавлять регуляризацию. Поддержаны три главные формы градиетного бустинга:

- Стандартный градиентный бустинг с возможностью изменения скорости обучения(learning rate).
- Стохастический градиентный бустинг с возможностью семплирования по строкам и колонкам датасета.
- Регуляризованный градиентный бустинг с L1 и L2 регуляризацией.


## Импорт пакетов
- `matplotlib.pyplot` - рисование графиков
- `pandas` - работа с таблицами
- `numpy` - работа с матрицами

In [1]:
import matplotlib.pyplot as plt

import pandas as pd

import numpy as np

%matplotlib inline

## Загрузка данных

`FuelConsumption.csv` (Рейтинги расхода топлива):

[Dataset source](https://open.canada.ca/data/en/dataset/98f1a129-f628-4ce4-b24d-6f16bf24dd64)

-   **MODELYEAR** e.g. 2014
-   **MAKE** e.g. Acura
-   **MODEL** e.g. ILX
-   **VEHICLE CLASS** e.g. SUV
-   **ENGINE SIZE** e.g. 4.7
-   **CYLINDERS** e.g 6
-   **TRANSMISSION** e.g. A6
-   **FUEL CONSUMPTION in CITY(L/100 km)** e.g. 9.9
-   **FUEL CONSUMPTION in HWY (L/100 km)** e.g. 8.9
-   **FUEL CONSUMPTION COMB (L/100 km)** e.g. 9.2
-   **CO2 EMISSIONS (g/km)** e.g. 182   --> low --> 0

In [2]:
import os
notebook_path = os.path.abspath("Notebook.ipynb")    # получить "точку отсчета" для поиска файла
data_csv = os.path.join(os.path.dirname(notebook_path), "data/FuelConsumptionCo2.csv")
df = pd.read_csv(data_csv)
df

,MODELYEAR,MAKE,MODEL,VEHICLECLASS,ENGINESIZE,CYLINDERS,TRANSMISSION,FUELTYPE,FUELCONSUMPTION_CITY,FUELCONSUMPTION_HWY,FUELCONSUMPTION_COMB,FUELCONSUMPTION_COMB_MPG,CO2EMISSIONS
0,2014,ACURA,ILX,COMPACT,2.0,4,AS5,Z,9.9,6.7,8.5,33,196
1,2014,ACURA,ILX,COMPACT,2.4,4,M6,Z,11.2,7.7,9.6,29,221
2,2014,ACURA,ILX HYBRID,COMPACT,1.5,4,AV7,Z,6.0,5.8,5.9,48,136
3,2014,ACURA,MDX 4WD,SUV - SMALL,3.5,6,AS6,Z,12.7,9.1,11.1,25,255
4,2014,ACURA,RDX AWD,SUV - SMALL,3.5,6,AS6,Z,12.1,8.7,10.6,27,244
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1062,2014,VOLVO,XC60 AWD,SUV - SMALL,3.0,6,AS6,X,13.4,9.8,11.8,24,271
1063,2014,VOLVO,XC60 AWD,SUV - SMALL,3.2,6,AS6,X,13.2,9.5,11.5,25,264
1064,2014,VOLVO,XC70 AWD,SUV - SMALL,3.0,6,AS6,X,13.4,9.8,11.8,24,271
1065,2014,VOLVO,XC70 AWD,SUV - SMALL,3.2,6,AS6,X,12.9,9.3,11.3,25,260


## Обучение модели (Train)

In [3]:
# разделить независимую и зависимую переменные / train и test

from sklearn.model_selection import train_test_split

y = df['CO2EMISSIONS'].copy()
X = df.drop(['CO2EMISSIONS', 'MAKE', 'MODEL', 'VEHICLECLASS', 'TRANSMISSION', 'FUELTYPE'], axis=1)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

### Параметры XGBoost (XGBoost Parameters)

---

#### Глобальная конфигурация (**Global Configuration**)
- Параметр **`verbosity`** может быть установлен в глобальной области с помощью `xgb.config_context()`: Детальность печатных сообщений.

    Допустимые значения: 
    - 0 (silent)
    - 1 (warning)
    - 2 (info)
    - 3 (debug)

---

#### Общие параметры (**General parameters**)
относятся к тому, какой booster используется для boosting, обычно это древовидная (tree) или линейная (linear) модель.

- `booster [default= gbtree]`: Какой booster использовать.

    Допустимые значения:
    - `gbtree`
    - `gblinear`
    - `dart`
    
    `gbtree` и `dart` используют модели на основе дерева (tree based models), в то время как `gblinear` использует линейные функции (linear functions).
    
- `verbosity [default=1]`: Детальность печатных сообщений.

    Допустимые значения: 
    - 0 (silent)
    - 1 (warning)
    - 2 (info)
    - 3 (debug)
    
    Иногда XGBoost пытается изменить конфигурации на основе эвристики, что отображается как предупреждающее сообщение. Если происходит неожиданное поведение, попробуйте увеличить значение подробности.
    
    Эвристический алгоритм (эвристика) — алгоритм решения задачи, включающий практический метод, не являющийся гарантированно точным или оптимальным, но достаточный для решения поставленной задачи. Позволяет ускорить решение задачи в тех случаях, когда точное решение не может быть найдено.

- `validate_parameters [default=True]`: Если установлено значение `True`, XGBoost будет выполнять проверку входных параметров, чтобы проверить, используется ли параметр или нет. *Эта функция все еще экспериментальная. Ожидается несколько ложных срабатываний.*
    
- `nthread [default=максимальное количество доступных потоков, если не установлено]`: Количество параллельных потоков, используемых для запуска XGBoost. *При выборе учитывайте конфликты потоков и гиперпоточность.*

- `disable_default_eval_metric [default='false']`: Используйте `'true'` (или `1`), чтобы отключить метрику по умолчанию.

- `num_pbuffer [устанавливается автоматически XGBoost, нет необходимости устанавливать пользователем]`: Размер буфера предсказания, обычно равный количеству обучающих экземпляров (training instances). Буферы используются для сохранения результатов прогноза последнего boosting шага.

- `num_feature [устанавливается автоматически XGBoost, нет необходимости устанавливать пользователем]`: Размер объекта, используемый при повышении, установлен на максимальный размер объекта.

---

#### Параметры booster'а (**Booster parameters**)
зависят от того какой booster выбран.

---

#### Параметры учебной задачи (**Learning task parameters**)
определяют сценарий обучения. Например, задачи регрессии (regression) могут использовать разные параметры с задачами ранжирования (ranking).

---

#### Параметры командной строки (**Command line parameters*)
относятся к поведению CLI-версии XGBoost.





- n_estimators — число деревьев.
- eta — размер шага. Предотвращает переобучение.
- gamma — минимальное изменение значения loss функции для разделения листа на поддеревья.
- max_depth — максимальная глубина дерева.
- lambda/alpha — L2/L1 регуляризация.

In [4]:
# импортировать пакет xgboost
import xgboost as xg

# создать объект XGBRegressor
xgb_r = xg.XGBRegressor(n_estimators = 10, max_depth=6)

# обучить модель
xgb_r.fit(X_train, y_train)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
             importance_type='gain', interaction_constraints='',
             learning_rate=0.300000012, max_delta_step=0, max_depth=6,
             min_child_weight=1, missing=nan, monotone_constraints='()',
             n_estimators=10, n_jobs=0, num_parallel_tree=1, random_state=0,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
             tree_method='exact', validate_parameters=1, verbosity=None)

## Оценка качества модели (Evaluation)

In [5]:
# использовать обученную модель для предсказания на test выборке
y_predicted = xgb_r.predict(X_test)

# вывести результаты предсказания
print(f'Variance score: {xgb_r.score(X_test, y_test)}')    # Coefficient of determination R^2 of the prediction
print(f"Residual sum of squares: {np.mean((y_predicted - y_test) ** 2)}")    # MSE

Variance score: 0.9475405274607717
Residual sum of squares: 213.62922278078167


In [6]:
from sklearn.metrics import r2_score

r2_score(y_true=y_test, y_pred=y_predicted)    # эквивалентно simple_regr.score(X_test, y_test)

0.9475405274607717

In [7]:
from sklearn.metrics import mean_squared_error

# MSE
mean_squared_error(y_test, y_predicted, squared=True)   # эквивалентно np.mean((y_predicted - y_test) ** 2)

213.62922278078167